In [14]:
import pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [3]:
bridges = pd.read_excel('data/BMMS_overview.xlsx')

In [4]:
bridges.head()

,road,km,type,LRPName,name,length,condition,structureNr,roadName,chainage,width,constructionYear,spans,zone,circle,division,sub-division,lat,lon,EstimatedLoc
0,N1,8.976,PC Girder Bridge,LRP008b,KANCHPUR PC GIRDER BRIDGE,397.0,C,101102,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,8.976,14.65,1986.0,8.0,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.704583,90.518833,bcs1
1,N1,10.543,Box Culvert,LRP010a,KATCHPUR BOX CULVERT,8.0,B,101106,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,10.543,21.70,1987.0,2.0,Dhaka,Dhaka,Narayanganj,Vitikandi,23.701528,90.528194,bcs1
2,N1,10.880,Box Culvert,LRP010b,NOYAPARA CULVERT,6.3,A,112531,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,10.880,12.20,1992.0,2.0,Dhaka,Dhaka,Narayanganj,Vitikandi,23.699861,90.530722,bcs1
3,N1,10.897,Box Culvert,LRP010c,ADUPUR CULVERT,6.3,A,112532,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,10.897,12.20,1984.0,2.0,Dhaka,Dhaka,Narayanganj,Vitikandi,23.697416,90.533777,bcs1
4,N1,11.296,Box Culvert,LRP011a,NAYABARI KASPUR BOX CULVERT,8.3,A,101110,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,11.296,21.45,1986.0,2.0,Dhaka,Dhaka,Narayanganj,Vitikandi,23.694361,90.537611,bcs1


Let's create a new ID using the road name and the LRP name to define a unique loaction point for the LRP so that bridges on equal LRP can be called uniquely. 

In [5]:
bridges['UniqueID']=bridges['road']+bridges['LRPName']

Now we need to look for duplicate bridges that may have the same UniqueID as two bridges can be constructed at the same location or a new bridge may be constructed next to an old one. For the purposes of this model, we chose to ignore these duplicated values. However, on observation of structureNr, we can see that all of these values are unique, and can be used to cross check and prevent other errors that may arise due to quality or capacity issues later.

In [6]:
boolen = bridges.duplicated(subset=['UniqueID'])
boolen.value_counts()

False    13044
Name: count, dtype: int64

In [7]:
boolen = bridges.duplicated(subset=['structureNr'])
boolen.value_counts()

False    13044
Name: count, dtype: int64

In [ ]:
# Let's handle the NaN values 

BMMS_mod = bridges.copy()
BMMS_mod['count_NaN'] = bridges[['road', 'km', 'type', 'LRPName', 'name', 'length', 'condition', 'structureNr', 'chainage', 'width', 'constructionYear', 'spans', 'lat', 'lon','UniqueID']].isnull().sum(axis=1)
BMMS_mod = BMMS_mod.sort_values(by=['road', 'LRPName', 'count_NaN', 'constructionYear'], ascending=[True, True, True, False])
BMMS_mod = BMMS_mod.reset_index(drop=True)
BMMS_mod = BMMS_mod.drop_duplicates(subset=['road','LRPName'], keep='first')
BMMS_mod = BMMS_mod.reset_index(drop=True)

BMMS_mod.shape

(13044, 22)

In [11]:
BMMS_mod.head()

,road,km,type,LRPName,name,length,condition,structureNr,roadName,chainage,...,spans,zone,circle,division,sub-division,lat,lon,EstimatedLoc,UniqueID,count_NaN
0,N1,8.976,PC Girder Bridge,LRP008b,KANCHPUR PC GIRDER BRIDGE,397.0,C,101102,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,8.976,...,8.0,Dhaka,Dhaka,Narayanganj,Narayanganj-1,23.704583,90.518833,bcs1,N1LRP008b,0
1,N1,10.543,Box Culvert,LRP010a,KATCHPUR BOX CULVERT,8.0,B,101106,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,10.543,...,2.0,Dhaka,Dhaka,Narayanganj,Vitikandi,23.701528,90.528194,bcs1,N1LRP010a,0
2,N1,10.880,Box Culvert,LRP010b,NOYAPARA CULVERT,6.3,A,112531,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,10.880,...,2.0,Dhaka,Dhaka,Narayanganj,Vitikandi,23.699861,90.530722,bcs1,N1LRP010b,0
3,N1,10.897,Box Culvert,LRP010c,ADUPUR CULVERT,6.3,A,112532,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,10.897,...,2.0,Dhaka,Dhaka,Narayanganj,Vitikandi,23.697416,90.533777,bcs1,N1LRP010c,0
4,N1,11.296,Box Culvert,LRP011a,NAYABARI KASPUR BOX CULVERT,8.3,A,101110,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,11.296,...,2.0,Dhaka,Dhaka,Narayanganj,Vitikandi,23.694361,90.537611,bcs1,N1LRP011a,0


Now that we have tentatively cleaned the bridges data from duplicate values, we need to see whether the LRP identifier for a bridge exists in the dataset for roads. However, since we are not looking at the location data yet, we can ignore the processing done for the accuracy and only focus on the LRPs and Road names

In [15]:
df = pd.read_excel('data/Roads_InfoAboutEachLRP.xlsx')
highList=pd.unique(df.road)
roadMap=pd.DataFrame()
atlas={}
for ID in highList:
    road = df.loc[df.road==ID]
    road.reset_index(drop=True) 
    road['Xchanged']='False'
    road['Ychanged']='False'
    atlas[ID]=len(road)
    if len(road)>5:
        for count in range(0,3):
            for i in road.index:
                # check if i is inside first or last five iterations and compute moving averages acordingly 
                while i in range(5, len(road)-5):
                    x_min, x_max = road.loc[i-5:i+5,'lon'].median() * .9999 , road.loc[i-5:i+5,'lon'].median() * 1.0001 # how big do we want to make search depth?
                    y_min, y_max = road.loc[i-5:i+5,'lat'].median() * .9999 , road.loc[i-5:i+5,'lat'].median() * 1.0001 

                    if not x_min <  road.loc[i,'lon'] < x_max: # check for outliers and overwrite 
                        road.loc[i,'lon'] = (road.loc[i-1,'lon'] + road.loc[i+1,'lon']) / 2 # position point right in between last and next point
                        road.loc[i,'Xchanged']='True'

                    if not y_min < road.loc[i,'lat'] < y_max:
                        road.loc[i,'lat'] = (road.loc[i-1,'lat'] + road.loc[i+1,'lat']) / 2 # position point right in between last and next point
                        road.loc[i,'Ychanged']='True'

                    i+=1
                    break 
    roadMap=pd.concat([roadMap,road])
roadMap=roadMap.reset_index(drop=1)

roadMap['UniqueID']=roadMap['road']+roadMap['lrp']

Now, using the UniqueID columns on each dataframe we can compare the two values to see if there are any points that are present in both datasets. This will allow is to see how many points may be lost if we merged the datasets right now.

In [16]:
lrpRoad=pd.unique(roadMap.UniqueID)
lrpBridge=pd.unique(bridges.UniqueID)
count=0
for i in lrpBridge:
	if i in lrpRoad:
		count+=1
print('Number of Coinciding bridges and Road LRPs is : '+str(count))
print('Number of Bridges is : '+str(len(lrpBridge)))
print('Number of Roads is : '+str(len(lrpRoad)))
print('Number of bridges without road LRPs : '+str(len(lrpBridge)-count))

Number of Coinciding bridges and Road LRPs is : 12561
Number of Bridges is : 13044
Number of Roads is : 51928
Number of bridges without road LRPs : 483


In [17]:
BMMS_mod = BMMS_mod.drop(columns=['count_NaN','UniqueID'])

In [19]:
BMMS_mod.to_excel('data/processed/BMMS_overview.xlsx', index=False, sheet_name='BMMS_overview')